**Import TensorFlow and other important libraries**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time

from IPython import display

load Cifar10 dataset  

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()
train_images=(train_images.reshape(train_images.shape[0],32,32,3).astype('float32'))
train_images=(train_images)/255
print(np.shape(train_images))
BUFFER_SIZE=50000
BATCH_SIZE=256

train_dataset=tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Generator Model Creation

In [ ]:
def generator_model():
  model=tf.keras.Sequential()
  model.add(layers.Dense(8*8*256,use_bias=False,input_shape=(100,)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Reshape((8,8,256)))
  
  model.add(layers.Conv2DTranspose(128,(5,5),strides=(2,2),padding='same',use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding='same',use_bias=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  model.add(layers.Conv2DTranspose(3,(5,5),strides=(1,1),padding='same',activation='sigmoid'))

  return model

**Sampling for Generator**

In [ ]:
generator=generator_model()
generator.summary()

**Creating some random noise to display**

In [ ]:
noise=tf.random.normal([1,100])
noise

**Display random noise image**

In [ ]:
generate_image=generator(noise,training=False)
plt.imshow(generate_image[0,:,:,0],cmap='gray')

**Discriminator model creation**

In [ ]:
def discriminator_model():
  model=tf.keras.Sequential()
  model.add(layers.Conv2D(128,(5,5),strides=(2,2),padding='same',input_shape=[32,32,3]))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout((0.4)))

  model.add(layers.Conv2D(64,(5,5),strides=(2,2),padding='valid'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(1))

  return model

**Discriminator Functionality**

In [ ]:
discriminator=discriminator_model()
decision=discriminator(generate_image)
print(decision)
discriminator.summary()



**Loss for Generator and Discriminator**

In [ ]:
cross_entropy=tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output,fake_output):
  real_loss=cross_entropy(tf.ones_like(real_output),real_output)
  fake_loss=cross_entropy(tf.zeros_like(fake_output),fake_output)
  total_loss=real_loss+fake_loss
  return total_loss

def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output),fake_output)

**Optimizer for Generator and Discriminator**

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

**Save Checkpoints**

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

**Experimental Setup**

In [ ]:
epoch=100
noise_dim=100
num_generate=10

seed=tf.random.normal([num_generate,noise_dim])

**Training Loop**

In [ ]:
@tf.function
def training(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape(persistent=True) as gen_tape, tf.GradientTape(persistent=True) as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [ ]:
def train(dataset,epoch):
  for epoch in range(epoch):
    start=time.time()

    for image_batch in dataset:
      training(image_batch)

    display.clear_output(wait=True)
    generate_save_image(generator,epoch+1,seed)

    if (epoch+1)%15==0:
      checkpoint.save(file_prefix=checkpoint_prefix)

    display.clear_output(wait=True)
    generate_save_image(generator,epoch,seed)

    

**Generate and save image**

In [ ]:
def generate_save_image(model,epoch,test_input):
  prediction=model(test_input,training=False)

  fig=plt.figure(figsize=(4,4))

  for i in range(prediction.shape[0]):
    plt.subplot(4,4,i+1)
    plt.imshow(prediction[i,:,:,0]*127.5+127.5,cmap='gray')
    plt.axis('off')

  plt.savefig('image at epoch_{:04d}.png'.format(epoch))
  plt.show()



**Train the model**

In [ ]:
%%time 
train(train_dataset,epoch)

**Restore the latest checkpoint**

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))